In [ ]:
%%capture
!pip install jsonlines
!pip install evaluate
!pip install bert_score
!unzip /content/model.zip

In [ ]:
import pickle

# To load back the saved data
with open('predictions_references.pkl', 'rb') as file:
    data = pickle.load(file)
    fined_predictions = data['fined_predictions']
    fined_references = data['fined_references']
    predictions = data['predictions']
    references = data['references']

EOFError: Ran out of input

In [ ]:
def custom_metric():
    succeed = 0
    correct = 0
    total_succeed_fail = 0
    total_correct_wrong = 0

    for prediction, reference in zip(predictions, references):
        try:
            reference = eval(reference)
            prediction = eval(prediction)
            keys = set(prediction.keys()) - set(["summary", "location", "specialization", "industry", "leadership_experience", "project", "team_experience", "other"])
            total_succeed_fail = len(predictions)
            total_correct_wrong = len(keys) * len(predictions)
            succeed += 1
            for key in keys:
                if reference[key] == prediction[key]:
                    correct += 1
                else:
                    print(key)
                    pass
        except:
            pass

    return float(f"{succeed/total_succeed_fail * 100:.2f}"), float(f"{correct/total_correct_wrong * 100:.2f}")

In [ ]:
import evaluate
import numpy as np

# Load metrics
bertscore_metric = evaluate.load("bertscore")

def compute_metrics(predictions, references):
    bertscore = bertscore_metric.compute(predictions=predictions, references=references, lang="en")
    bertscore = {key: np.mean(bertscore[key]) for key in bertscore if key != "hashcode"}
    succeed, correct = custom_metric()

    return {
        "bertscore": bertscore,
        "succeed": succeed,
        "correct": correct,
        }

In [ ]:
fined_results = compute_metrics(predictions=fined_predictions, references=fined_references)
results = compute_metrics(predictions=predictions, references=references)

NameError: name 'compute_metrics' is not defined

In [ ]:
from pprint import pprint
print("fined_results:\n\n")
pprint(fined_results)
print("\n\nresults:\n\n")
pprint(results)

In [ ]:
import json

# Save to file
with open('fined_results.json', 'w') as file:
    json.dump(fined_results, file)

with open('results.json', 'w') as file:
    json.dump(results, file)